In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.support.ui import Select

from fuzzywuzzy import fuzz
from time import sleep
import operator

C:\Users\arvind.baranwal\AppData\Local\Continuum\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
def clean(name):
    name = name.lower()
    junk_words = [" corporation"," inc"," ltd"," pvt","."," llc"," llp","&amp",";",","]
    for word in junk_words:
        name = name.replace(word,"")
    name = name.strip()
    return name

In [4]:
def get_browser():
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('log-level=3')
    browser = webdriver.Chrome(chrome_options=chrome_options)
    return browser

In [112]:
import pandas as pd
data = pd.read_excel('DecRenewalAccounts_V.08_ReportID.xlsx',sheet_name = 'Intellect DEC ReportID')
data.shape

(292, 17)

In [113]:
data.head(2)

,STAG Report ID,Unique ID,Account_Name2,Midway Address 1,Midway Address 2,Midway City,Midway State,Midway Zip,LOB3,Renewal Date4,Cleaned Updated Account_Name,Cleaned Updated Midway Address,Cleaned Updated Midway City,Cleaned Updated Midway State,Cleaned Updated Midway Zip,LOB39,Renewal Date410
0,c0003466921371,DEC19REN387,MASSACHUSETTS HOUSING FINANCE AGENCY,1 BEACON ST STE 2700,NaN,BOSTON,MA,2108,WC Lines,2019-12-05,MASSACHUSETTS HOUSING FINANCE AGENCY,1 BEACON ST STE 2700,BOSTON,MA,2108,WC Lines,2019-12-05
1,c0000837722019,DEC19REN675,FLEETWOOD ALUMINUM PRODUCTS INC,"1 FLEETWOOD WAY, P.O.BOX 1086",NaN,CORONA,CA,92879,WC Lines,2019-12-07,FLEETWOOD ALUMINUM PRODUCTS INC,1 FLEETWOOD WAY,CORONA,CA,92879,WC Lines,2019-12-07


In [114]:
query_company = data['Cleaned Updated Account_Name'].tolist()
original = data['Cleaned Updated Account_Name'].tolist()

In [115]:
len(query_company)

292

In [117]:
# driver= webdriver.Chrome('C:/Users/deepti.patil/Downloads/chromedriver.exe')
driver = get_browser()

url = "https://www.careerbliss.com"

# query_company = ['at&t','cortera','zulily']


company_name =[]
review_link =[]
score_max =[]
original_name=[]

for i in range(0,len(query_company)):
    
#     driver.implicitly_wait(3)
    
    driver.get(url)
    
    #selecting "Companies" tab from the drop down menu
    driver.find_element_by_xpath("""//*[@id="type-dropdown"]/span/b""").click()
    driver.find_element_by_xpath("""//*[@id="type-dropdown"]/ul/li[4]/a""").click()

    co_name = driver.find_element_by_xpath("""//*[@id="search-q"]""")
    co_name.send_keys(query_company[i])

    ##submitting
    driver.find_element_by_xpath("""//*[@id="search-icon"]""").click()
    
    sleep(2)

    soup = BeautifulSoup(driver.page_source, "html.parser")

    try:

        ### searching for names on the first page only.
        name =[]
        for n in soup.find_all("a",{"class":"large company-name ng-binding"}):
            name.append(n.text.strip())
        
        print(name)

        ## storing list of companies fuzzy score with the query company name
        score=[]
        for s in name:
            score.append(fuzz.ratio(clean(query_company[i]).lower(),clean(s).lower()))
            
        print(score)

        ## cut off as 90-- if more than one cutoff is 90-- choosing max value. What if max value appears more than one time??
        values={}
        for k,v in enumerate(score):
            if v>79:
                values[k] = v

        position = max(values.items(), key=operator.itemgetter(1))[0]
        score_maxvalue = max(values.items(), key=operator.itemgetter(1))[1]
        
        
        ## appending with 
        NAME = name[position]
        ORI_NAME = original[i]
        LINK = "https://www.careerbliss.com"+soup.find_all("a",{"class":"large company-name ng-binding"},href=True)[position]['href']+"reviews/"
        SCORE= score_maxvalue
        
        
    except:
        NAME=""
        ORI_NAME = original[i]
        LINK=""
        SCORE= ""

    company_name.append(NAME)
    review_link.append(LINK)
    score_max.append(SCORE)
    original_name.append(ORI_NAME)
    
driver.quit()
pd.DataFrame({'company_name':company_name,'original_name':original_name,'review_link':review_link,'max_score':score_max}).to_excel('Dec_Renewal_careerbliss.xlsx')        

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


['Minnesota Housing Finance Agency', 'DC Housing Finance Agency', 'California Housing Finance Agency', 'United Guaranty', 'Thermo Fisher Scientific', 'PAREXEL International', 'Agilent Technologies', 'Liberty Mutual']
[76, 79, 72, 31, 37, 32, 32, 20]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['A.O. Smith']
[29]
['Waterford Hotel Group, Inc.', 'Kimpton Hotel & Restaurant Group']
[67, 55]
[]
[]
[]
[]
[]
[]
[]
[]
['Central Metal Products', 'Central Metal Finishing', 'Gerdau AmeriSteel']
[78, 76, 52]
[]
[]
[]
[]
['Westmont Industries', 'Allied Van Lines']
[100, 40]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['Federal Management Systems', 'Education Management Solutions', 'Symphony Services', 'Maxim Healthcare Services', 'Amcat Adjusting Services', 'BearingPoint', 'Entertainment Partners', 'Mustang Engineering', 'The Active Network', 'Delaware North Cos.']
[62, 61, 51, 55, 52, 32, 50, 40, 32, 41]
[]
[]
[]
[]
['DeVry University', 'American Electric Power', 'QuinStreet']
[100, 31, 31]
[]
[]
[]
[]
['National Envelope',

[]
[]
[]
[]
[]
[]
[]
[]
['Ric-Man Construction', 'Black & Veatch', 'Tradesmen International']
[70, 25, 49]
['Weber Group', 'Voice Technologies Group', 'BT Conferencing', 'AWS Convergence Technologies', 'Sun Microsystems', 'TriZetto', 'Applied Materials', 'Meditech Communications', 'Primerica', 'Echo Global Logistics']
[100, 46, 31, 26, 22, 21, 21, 29, 20, 25]
[]
[]
[]
[]
[]
[]
[]
[]
['Paragon Marketing Group', 'Vayan Marketing Group', 'Lorel Marketing Group', 'Air Marketing Group', 'Paragon Technology Group', 'Morex Marketing', 'Phoenix Marketing Solutions', 'Acquity Group', 'Chrysler', 'Examiner.com']
[100, 86, 77, 86, 72, 58, 64, 44, 19, 35]
[]
[]
['Yazaki', 'Brand Services', 'TransUnion', 'MTD Products', 'Duke Energy', 'Alpha Technologies', 'Adams Thermal Systems', 'Advanced Energy', 'First Solar', 'Kimco Realty']
[19, 41, 23, 32, 28, 33, 30, 30, 33, 38]
[]
[]
[]
[]
['USA Mobility', 'Brightpoint', 'Ronco', 'T-Mobile', 'Christie Digital Systems']
[28, 14, 9, 16, 34]
[]
[]
['Applied E

In [108]:
import pandas as pd
pd.DataFrame({'company_name':company_name,'original_name':original_name,'review_link':review_link,'max_score':score_max}).to_excel('Nov_Renewal_careerbliss.xlsx')

## built condensed function for getting all matches

In [5]:


def get_browser():
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('log-level=3')
    browser = webdriver.Chrome(chrome_options=chrome_options)
    return browser

def clean(name):
    name = name.lower()
    junk_words = [" corporation"," inc"," ltd"," pvt","."," llc"," llp","&amp",";",","]
    for word in junk_words:
        name = name.replace(word,"")
    name = name.strip()
    return name

driver = get_browser()

def candidates_careerbliss_1(companyname,match_cutoff):
    
    driver = get_browser()
    url = "https://www.careerbliss.com"

    driver.get(url)
    driver.find_element_by_xpath("""//*[@id="type-dropdown"]/span/b""").click()
    driver.find_element_by_xpath("""//*[@id="type-dropdown"]/ul/li[4]/a""").click()

    co_name = driver.find_element_by_xpath("""//*[@id="search-q"]""")
    co_name.send_keys(companyname.lower())

    ##submitting
    driver.find_element_by_xpath("""//*[@id="search-icon"]""").click()
    
    sleep(2)

    soup = BeautifulSoup(driver.page_source, "html.parser")
    try:

        ### searching for names on the first page only.
        name =[]
        for n in soup.find_all("a",{"class":"large company-name ng-binding"}):
            name.append(n.text.strip())

        ## storing list of companies fuzzy score with the query company name
        scores=[]
        found_names = []
        review_links = []
        position = 0
        for s in name:
            match_score = fuzz.ratio(clean(companyname).lower(),clean(s).lower()) 
            if match_score > match_cutoff:
                found_names.append(s)
                review_links.append("https://www.careerbliss.com"+soup.find_all("a",{"class":"large company-name ng-binding"},href=True)[position]['href']+"reviews/")
                scores.append(match_score)
            position+=1             
        #         print('names after matching cut-off')
        #         print(len(found_names))
        #         print(found_names)
        #         print('review links')
        #         print(len(review_links))
        #         print(review_links)
        #         print('matching scores')
        #         print(len(scores))
        #         print(scores)

    except:
        found_names = []
        review_links = []
        scores=[]  
    
    driver.refresh 
    return found_names, review_links, scores


C:\Users\arvind.baranwal\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: use options instead of chrome_options
  import sys


In [6]:
candidates_careerbliss_1('Minnesota Housing Finance Agency',0)

C:\Users\arvind.baranwal\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: use options instead of chrome_options
  import sys


(['Minnesota Housing Finance Agency',
  'DC Housing Finance Agency',
  'California Housing Finance Agency',
  'Digi',
  'Dot Hill Systems',
  'United Guaranty',
  'Agilent Technologies'],
 ['https://www.careerbliss.com/minnesota-housing-finance-agency/reviews/',
  'https://www.careerbliss.com/dchfa/reviews/',
  'https://www.careerbliss.com/california-housing-finance-agency/reviews/',
  'https://www.careerbliss.com/digi/reviews/',
  'https://www.careerbliss.com/dot-hill-systems/reviews/',
  'https://www.careerbliss.com/united-guaranty/reviews/',
  'https://www.careerbliss.com/agilent-technologies/reviews/'],
 [100, 81, 80, 11, 25, 26, 19])